# Music recommender system

One of the most used machine learning algorithms is recommendation systems. A **recommender** (or recommendation) **system** (or engine) is a filtering system which aim is to predict a rating or preference a user would give to an item, eg. a film, a product, a song, etc.
  
> Content-based filters predicts what a user likes based on what that particular user has liked in the past. On the other hand, collaborative-based filters predict what a user like based on what other users, that are similar to that particular user, have liked.

By:
### RA1911003010260	Ashish Kashyap
### RA1911003010266	Prakhar Singh
### RA1911003010264	Akshat Saxena
### RA1911003010257	Aaryan Singhal
### RA1911003010276	Shivansh Sinha
### RA1911003010280	Ansh Thapliyal

### Importing required libraries

First, we'll import all the required libraries.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset

In [4]:
songs = pd.read_csv('../input/datasong/songdata.csv')

In [5]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Because of the dataset being so big, we are going to resample only 5000 random songs.

In [6]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

We can notice also the presence of `\n` in the text, so we are going to remove it.

In [7]:
count =1
while count<1: 
    songs['text'] = songs['text'].str.replace(r'\n', '')
    count=count+1

In [8]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [9]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

*How do we use this matrix for a recommendation?* 

We now need to calculate the similarity of one lyric to another. We are going to use **cosine similarity**.

We want to calculate the cosine similarity of each item with every other item in the dataset. So we just pass the lyrics_matrix as argument.

In [10]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [11]:
similarities = {}

In [12]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

After that, all the magic happens. We can use that similarity scores to access the most similar items and give a recommendation.

For that, we'll define our Content based recommender class.

In [13]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

Now, instantiate class

In [14]:
recommedations = ContentBasedRecommender(similarities)

Then, we are ready to pick a song from the dataset and make a recommendation.

In [15]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [16]:
recommedations.recommend(recommendation)

The 4 recommended songs for You Took Him Off My Hands are:
Number 1:
Took My Love by Pitbull with 0.324 similarity score
--------------------
Number 2:
Out Of My Hands by Jason Mraz with 0.298 similarity score
--------------------
Number 3:
Don't Leave Me Now by Supertramp with 0.232 similarity score
--------------------
Number 4:
All Of Me by Diana Ross with 0.225 similarity score
--------------------


And we can pick another random song and recommend again:

In [17]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [18]:
recommedations.recommend(recommendation2)

The 4 recommended songs for That Old Feeling are:
Number 1:
Give Me Back That Old Familiar Feeling by Glen Campbell with 0.309 similarity score
--------------------
Number 2:
Back When The Old Homeplace Was New by Tom T. Hall with 0.292 similarity score
--------------------
Number 3:
Manhattan Moon by Kirsty Maccoll with 0.269 similarity score
--------------------
Number 4:
Old Flame by Alabama with 0.264 similarity score
--------------------
